In [3]:
import argparse
import os
import sys
from pathlib import Path

import torch
import torch.backends.cudnn as cudnn

from utils.dataloaders import IMG_FORMATS, VID_FORMATS, LoadImages, LoadStreams
from utils.general import (LOGGER, check_file, check_img_size, check_imshow, check_requirements, colorstr, cv2,
                           increment_path, non_max_suppression, print_args, scale_coords, strip_optimizer, xyxy2xywh)
from utils.plots import Annotator, colors, save_one_box
from utils.torch_utils import select_device, time_sync

# путь до сохранения артефактов
DETECT_IMAGES = 'files_obr/'
# path images
SAVE_IMAGES = 'files_obr/save_images/'
# загрузка словаря замен
dict_synth = {
    'fissure': 0, 
    'large_pit': 1, 
    'small_pit': 2, 
    'patch': 3
}

@torch.no_grad()
def detect(
    source,
    model,
    imgsz=(1280, 1280),  # inference size (height, width)
    conf_thres=0.3,  # confidence threshold
    iou_thres=0.4,  # NMS IOU threshold
    max_det=1000,  # maximum detections per image
    device='cpu',  # cuda device, i.e. 0 or 0,1,2,3 or cpu
    view_img=False,  # show results
    save_txt=True,  # save results to *.txt
    save_conf=False,  # save confidences in --save-txt labels
    save_crop=False,  # save cropped prediction boxes
    nosave=False,  # do not save images/videos
    classes=None,  # filter by class: --class 0, or --class 0 2 3
    agnostic_nms=False,  # class-agnostic NMS
    augment=False,  # augmented inference
    visualize=True,  # visualize features
    update=False,  # update all models
    line_thickness=3,  # bounding box thickness (pixels)
    hide_labels=False,  # hide labels
    hide_conf=True,  # hide confidences
    half=False,  # use FP16 half-precision inference
    dnn=False,  # use OpenCV DNN for ONNX inference
):
    """
    Функция для представления результата.
    """
    source = str(source)
    save_img = not nosave and not source.endswith('.txt')  # save inference images
    # Directories
    save_dir = increment_path(DETECT_IMAGES, True)
    (save_dir / 'labels' if save_txt else save_dir).mkdir(parents=True, exist_ok=True)  # make dir
    # 
    save_path_origin = str(save_dir/ 'images' / source)
    
    # формирование изображения
    stride, names, pt = model.stride, model.names, model.pt
    imgsz = check_img_size(imgsz, s=stride)  # check image size

    dataset = LoadImages(source, img_size=imgsz, stride=stride)
    print(dataset)
    bs = 1  # batch_size
    vid_path, vid_writer = [None] * bs, [None] * bs
    vid_path, vid_writer = [None] * bs, [None] * bs

    # Run inference
    model.warmup(imgsz=(1 if pt else bs, 3, *imgsz))  # warmup
    dt, seen = [0.0, 0.0, 0.0], 0
    all_labels = []
    for path, im, im0s, vid_cap, s in dataset:
        t1 = time_sync()
        im = torch.from_numpy(im).to(device)
        im = im.half() if model.fp16 else im.float()  # uint8 to fp16/32
        im /= 255  # 0 - 255 to 0.0 - 1.0
        if len(im.shape) == 3:
            im = im[None]  # expand for batch dim
        t2 = time_sync()
        dt[0] += t2 - t1

        # Inference
        visualize = increment_path(DETECT_IMAGES, True) 
        pred = model(im, augment=augment, visualize=False)
        t3 = time_sync()
        dt[1] += t3 - t2

        # NMS
        print(pred)
        pred = non_max_suppression(pred, conf_thres, iou_thres, classes, agnostic_nms, max_det=max_det)
        dt[2] += time_sync() - t3
        print(pred)
        # Process predictions
        for i, det in enumerate(pred):  # per image
            seen += 1
            p, im0, frame = path, im0s.copy(), getattr(dataset, 'frame', 0)
            p = Path(p)  # to Path
            save_path = str(save_dir/ 'detection_images' / p.name)  # im.jpg
            #save_path_origin = str(save_dir/ 'images' / p.name)
            txt_path = str(save_dir / 'labels' / p.stem) + ('' if dataset.mode == 'image' else f'_{frame}')  # im.txt
            s += '%gx%g ' % im.shape[2:]  # print string
            gn = torch.tensor(im0.shape)[[1, 0, 1, 0]]  # normalization gain whwh
            imc = im0.copy() if save_crop else im0  # for save_crop
            # сохранение исходного изображения
            #cv2.imwrite(save_path_origin, im0)
            annotator = Annotator(im0, line_width=line_thickness, pil=True, example=str(names))
            if len(det):
                # Rescale boxes from img_size to im0 size
                det[:, :4] = scale_coords(im.shape[2:], det[:, :4], im0.shape).round()
                # Print results
                for c in det[:, -1].unique():
                    n = (det[:, -1] == c).sum()  # detections per class
                    s += f"{n} {names[int(c)]}{'s' * (n > 1)}, "  # add to string

                # Write results
                for *xyxy, conf, cls in reversed(det):
                    if save_txt:  # Write to file
                        xywh = (xyxy2xywh(torch.tensor(xyxy).view(1, 4)) / gn).view(-1).tolist()  # normalized xywh
                        line = (cls, *xywh, conf) if save_conf else (cls, *xywh)  # label format
                        file_check_txt = txt_path + '.txt'
                        with open(file_check_txt, 'a') as f:
                            f.write(('%g ' * len(line)).rstrip() % line + '\n')

                    if save_img or save_crop or view_img:  # Add bbox to image
                        c = int(cls)  # integer class
                        label = None if hide_labels else (names[c] if hide_conf else f'{names[c]} {conf:.2f}')
                        label = label.replace('dustbin', 'rubbish').replace('graffiti', 'advertising')
                        label_id = dict_synth[label]
                        all_labels.append(label_id)
                        annotator.box_label(xyxy, label, color=colors(c, True))
                        if save_crop:
                            save_one_box(xyxy, imc, file=save_dir / 'crops' / names[c] / f'{p.stem}.jpg', BGR=True)

            # Stream results
            im0 = annotator.result()
            if view_img:
                cv2.imshow(str(p), im0)
                cv2.waitKey(1)  # 1 millisecond

            # Save results (image with detections)
            if save_img:
                if dataset.mode == 'image':
                    cv2.imwrite(save_path, im0)
                else:  # 'video' or 'stream'
                    if vid_path != save_path:  # new video
                        vid_path = save_path
                        if isinstance(vid_writer, cv2.VideoWriter):
                            vid_writer.release()  # release previous video writer
                        if vid_cap:  # video
                            fps = vid_cap.get(cv2.CAP_PROP_FPS)
                            w = int(vid_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
                            h = int(vid_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
                        else:  # stream
                            fps, w, h = 30, im0.shape[1], im0.shape[0]
                            save_path += '.mp4'
                        vid_writer = cv2.VideoWriter(save_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (w, h))
                    vid_writer.write(im0)
        # Print time (inference-only)
        LOGGER.info(f'{s}Done. ({t3 - t2:.3f}s)')

    # Print results
    t = tuple(x / seen * 1E3 for x in dt)  # speeds per image
    LOGGER.info(f'Speed: %.1fms pre-process, %.1fms inference, %.1fms NMS per image at shape {(1, 3, *imgsz)}' % t)
    if save_txt or save_img:
        s = f"\n{len(list(save_dir.glob('labels/*.txt')))} labels saved to {save_dir / 'labels'}" if save_txt else ''
        LOGGER.info(f"Results saved to {colorstr('bold', save_dir)}{s}")
    if update:
        strip_optimizer(weights)  # update model (to fix SourceChangeWarning)
        
    return all_labels

In [4]:

from models.common import DetectMultiBackend
from utils.torch_utils import select_device

# Load model
device = 'cpu'
# загрузка модели
device = select_device(device)
model_cv = DetectMultiBackend('C:/Users/Анна/Desktop/Roads_project/Roads/yolov5/runs/train/exp/weights/yolo_by_roads_v4.pt', device=device, dnn=False, fp16=False)

# видео
new_file = 'for_tracking_tests' + '.mp4' # ВСЕГДА В КОНЦЕ ДОЛЖНО БЫТЬ РАСШИРЕНИЕ МП4
with torch.no_grad():
    all_labels = detect(new_file, model_cv)


YOLOv5  2022-8-4 Python-3.8.9 torch-1.12.1+cpu CPU

Fusing layers... 
Model summary: 213 layers, 7020913 parameters, 0 gradients


video 1/1 (1/590) C:\Users\Анна\Desktop\Roads_project\Roads\yolov5\for_tracking_tests.mp4: 736x1280 1 large_pit, Done. (0.739s)


tensor([[[5.07478e+00, 3.84110e+00, 1.13329e+01,  ..., 5.79258e-02, 1.56464e-01, 4.88118e-02],
         [1.31127e+01, 4.37240e+00, 2.62743e+01,  ..., 8.29889e-02, 8.23497e-02, 2.99626e-02],
         [2.16633e+01, 4.40853e+00, 3.68172e+01,  ..., 7.59848e-02, 7.06259e-02, 2.53337e-02],
         ...,
         [1.18981e+03, 7.22848e+02, 4.99502e+01,  ..., 9.61916e-01, 6.09811e-02, 5.79022e-03],
         [1.24253e+03, 7.03872e+02, 3.06883e+01,  ..., 9.76614e-01, 4.69240e-02, 5.33756e-03],
         [1.26644e+03, 6.97134e+02, 2.29209e+01,  ..., 9.75999e-01, 4.59910e-02, 6.97561e-03]]])
[tensor([[220.77888, 362.79608, 236.40456, 401.43146,   0.40480,   1.00000]])]


video 1/1 (2/590) C:\Users\Анна\Desktop\Roads_project\Roads\yolov5\for_tracking_tests.mp4: 736x1280 2 fissures, Done. (0.864s)


tensor([[[4.27951e+00, 4.81753e+00, 1.06754e+01,  ..., 5.48449e-02, 1.32472e-01, 5.30691e-02],
         [1.31659e+01, 5.08908e+00, 2.77154e+01,  ..., 7.98790e-02, 5.92592e-02, 4.25294e-02],
         [2.08050e+01, 4.44220e+00, 3.47215e+01,  ..., 7.56741e-02, 5.20326e-02, 3.89795e-02],
         ...,
         [1.22314e+03, 7.13274e+02, 3.02590e+01,  ..., 9.64496e-01, 5.59728e-02, 8.59657e-03],
         [1.23072e+03, 7.11640e+02, 2.65239e+01,  ..., 9.59933e-01, 6.54999e-02, 1.29187e-02],
         [1.27162e+03, 7.14850e+02, 2.27391e+01,  ..., 9.43693e-01, 9.52230e-02, 1.26413e-02]]])
[tensor([[4.15096e+02, 6.74404e+02, 4.56028e+02, 7.24585e+02, 6.41982e-01, 0.00000e+00],
        [4.40553e+02, 3.68304e+02, 5.87998e+02, 3.77482e+02, 3.10759e-01, 0.00000e+00]])]


...

video 1/1 (18/590) C:\Users\Анна\Desktop\Roads_project\Roads\yolov5\for_tracking_tests.mp4: 736x1280 1 fissure, 1 small_pit, 1 patch, Done. (0.971s)


tensor([[[2.89246e+00, 5.98841e+00, 5.92441e+00,  ..., 4.02565e-01, 1.36610e-01, 3.55316e-02],
         [1.20755e+01, 5.01556e+00, 2.33747e+01,  ..., 6.37099e-01, 7.49184e-02, 3.69097e-02],
         [2.18321e+01, 4.16954e+00, 3.50421e+01,  ..., 4.47261e-01, 1.49573e-01, 2.32566e-02],
         ...,
         [1.19596e+03, 7.27703e+02, 6.05777e+01,  ..., 9.32425e-01, 1.34881e-01, 6.96789e-03],
         [1.24011e+03, 7.24830e+02, 4.16666e+01,  ..., 9.22256e-01, 1.54060e-01, 7.77866e-03],
         [1.27515e+03, 7.13221e+02, 1.68846e+01,  ..., 9.13273e-01, 1.07937e-01, 9.21149e-03]]])
[tensor([[3.75488e+02, 6.82833e+02, 4.80838e+02, 7.30813e+02, 8.23146e-01, 3.00000e+00],
        [2.68450e+02, 4.49868e+02, 5.20944e+02, 4.72290e+02, 4.21944e-01, 0.00000e+00],
        [1.07958e+03, 2.72984e+02, 1.09228e+03, 2.79689e+02, 3.05408e-01, 2.00000e+00]])]


video 1/1 (19/590) C:\Users\Анна\Desktop\Roads_project\Roads\yolov5\for_tracking_tests.mp4: 736x1280 1 fissure, 1 small_pit, 1 patch, Done. (0.912s)


tensor([[[2.88544e+00, 5.99798e+00, 5.91542e+00,  ..., 4.01248e-01, 1.36496e-01, 3.56597e-02],
         [1.20427e+01, 5.01822e+00, 2.33422e+01,  ..., 6.36344e-01, 7.48161e-02, 3.68500e-02],
         [2.18345e+01, 4.16857e+00, 3.50469e+01,  ..., 4.46243e-01, 1.49656e-01, 2.33154e-02],
         ...,
         [1.19603e+03, 7.27697e+02, 6.05792e+01,  ..., 9.32432e-01, 1.34321e-01, 6.99356e-03],
         [1.24011e+03, 7.25019e+02, 4.18722e+01,  ..., 9.22326e-01, 1.53741e-01, 7.79757e-03],
         [1.27516e+03, 7.13276e+02, 1.68934e+01,  ..., 9.13082e-01, 1.07777e-01, 9.23941e-03]]])
[tensor([[3.75416e+02, 6.82812e+02, 4.80829e+02, 7.30840e+02, 8.23431e-01, 3.00000e+00],
        [2.68656e+02, 4.49896e+02, 5.21084e+02, 4.72285e+02, 4.21190e-01, 0.00000e+00],
        [1.07958e+03, 2.72984e+02, 1.09228e+03, 2.79688e+02, 3.06253e-01, 2.00000e+00]])]


KeyboardInterrupt: 

In [5]:
import os
os.getcwd()

'C:\\Users\\DarkSidius\\Downloads\\Telegram Desktop\\yolov5-master\\yolov5-master'